# 0 - Installation & Imports

In [ ]:
%%time
!pip install -q networkx
!pip install -q networkit
!pip install -q ogb
!pip install -q torch_geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [ ]:
from os import mkdir,path
import numpy
from pandas import DataFrame
from tqdm.notebook import tqdm
from IPython.display import clear_output


from torch import serialization
from torch_geometric.data.storage import GlobalStorage
from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
serialization.add_safe_globals([GlobalStorage, DataEdgeAttr, DataTensorAttr])
from ogb.linkproppred import PygLinkPropPredDataset

from torch_geometric.utils import to_networkx

import networkit as nk

# 1 - Data Loading and Preparation

In [ ]:
dataset = PygLinkPropPredDataset(name='ogbl-ddi')
clear_output()
graph_data = dataset[0]

In [ ]:
Grpah_nx = to_networkx(graph_data).to_undirected()
Graph = nk.nxadapter.nx2nk(Grpah_nx, weightAttr=None)
Graph

In [30]:
features_save_path = "Features - Networkit"
if not path.exists(features_save_path):
    mkdir(features_save_path)

# 2 - Computing Link Prediction Features

## 2.1 - Common Neighbors Index

In [ ]:
Common_Neighbors_indx = nk.linkprediction.CommonNeighborsIndex(Graph)
records = []
for u, v in tqdm(Graph.iterEdges() , total=Graph.numberOfEdges()):
    records.append( [int(u),int(v),Common_Neighbors_indx.run(u,v) ] )

df = DataFrame.from_records(records , columns=['node_u','node_v','CommonNeighborsIndex'])
df.to_csv( path.join(features_save_path,'Common Neighbors Index.csv'),index=False)
df.sample(4)

  0%|          | 0/1067911 [00:00<?, ?it/s]

,node_u,node_v,CommonNeighborsIndex
929200,2389,2898,243.0
45363,126,1034,702.0
194750,327,1120,629.0
889278,2225,2274,123.0


## 2.2 - Same Community Index

In [ ]:
Same_Community_indx = nk.linkprediction.SameCommunityIndex(Graph)
records = []
for u, v in tqdm(Graph.iterEdges() , total=Graph.numberOfEdges()):
    records.append( [int(u),int(v),Same_Community_indx.run(u,v) ] )

df = DataFrame.from_records(records , columns=['node_u','node_v','SameCommunityIndex'])
df.to_csv( path.join(features_save_path,'Same Community Index.csv'),index=False)
df.sample(4)

  0%|          | 0/1067911 [00:00<?, ?it/s]

,node_u,node_v,SameCommunityIndex
809623,1795,4034,0.0
395842,630,4112,1.0
907592,2299,3831,1.0
187830,316,3943,1.0


## 2.3 - Resource Allocation Index

In [ ]:
Resource_Allocation_indx = nk.linkprediction.ResourceAllocationIndex(Graph)
records = []
for u, v in tqdm(Graph.iterEdges() , total=Graph.numberOfEdges()):
    records.append( [int(u),int(v),Resource_Allocation_indx.run(u,v) ] )

df = DataFrame.from_records(records , columns=['node_u','node_v','ResourceAllocationIndex'])
df.to_csv( path.join(features_save_path,'Resource Allocation Index.csv'),index=False)
df.sample(4)

  0%|          | 0/1067911 [00:00<?, ?it/s]

,node_u,node_v,ResourceAllocationIndex
1065571,4054,4261,0.516192
305328,488,652,0.876634
590345,1004,1624,0.549746
973685,2617,3876,0.446840


# 3 - Computing Centrality of Nodes

## 3.1 - Approx Betweenness

In [ ]:
%%time
Aprx_BTWN = nk.centrality.ApproxBetweenness(Graph, epsilon=0.1).run()

CPU times: user 12.6 s, sys: 16.8 ms, total: 12.6 s
Wall time: 12.7 s


## 3.2 - Estimate Betweenness

In [ ]:
%%time
Est_BTWN = nk.centrality.EstimateBetweenness(Graph,300).run()

CPU times: user 15.5 s, sys: 14 ms, total: 15.6 s
Wall time: 15.6 s


## 3.3 - Approx Closeness

In [ ]:
%%time
Aprx_CLSNS = nk.centrality.ApproxCloseness(Graph, 300).run()

CPU times: user 1.29 s, sys: 5 ms, total: 1.29 s
Wall time: 1.29 s


## 3.4 - Approx Electrical Closeness

In [ ]:
%%time
Aprx_ELC_CLSNS = nk.centrality.ApproxElectricalCloseness(Graph, eps=0.1, kappa=0.3).run()

CPU times: user 21.4 s, sys: 214 ms, total: 21.6 s
Wall time: 21.7 s


## 3.5 - Degree Centrality

In [ ]:
%%time
DGR_CNTR = nk.centrality.DegreeCentrality(Graph).run()

CPU times: user 72 µs, sys: 1 µs, total: 73 µs
Wall time: 77.5 µs


## 3.6 - K Path Centrality

In [ ]:
%%time
K_PTH_CNTR = nk.centrality.KPathCentrality(Graph, alpha=0.2, k=0).run()

CPU times: user 15.7 s, sys: 30 ms, total: 15.8 s
Wall time: 15.8 s


## 3.7 - Katz Centrality

In [ ]:
%%time
KTZ_CNTR = nk.centrality.KatzCentrality(Graph, alpha=0, beta=0.1, tol=1e-8).run()

CPU times: user 117 ms, sys: 1.97 ms, total: 119 ms
Wall time: 134 ms


## 3.8 - Laplacian Centrality

In [ ]:
%%time
LPLC_CNTR = nk.centrality.LaplacianCentrality(Graph).run()

CPU times: user 34.3 ms, sys: 0 ns, total: 34.3 ms
Wall time: 43.1 ms


## 3.9 - SciPy EVZ

In [ ]:
%%time
SP_EVZ = nk.centrality.SciPyEVZ(Graph).run()

CPU times: user 5.17 s, sys: 91.8 ms, total: 5.26 s
Wall time: 5.35 s


## 3.10 - Harmonic Closeness

In [ ]:
%%time
HRM_CLSNS = nk.centrality.HarmonicCloseness(Graph).run()

CPU times: user 2min 4s, sys: 177 ms, total: 2min 5s
Wall time: 2min 5s


## 3.11 - Saving Nodes Features

In [ ]:
df = DataFrame({'Node':range(Graph.numberOfNodes()),'Est_BTWN':Est_BTWN.scores(), 'Aprx_BTWN':Aprx_BTWN.scores(),
                'Aprx_CLSNS':Aprx_CLSNS.scores(), 'Aprx_ELC_CLSNS':Aprx_ELC_CLSNS.scores(),
                'DGR_CNTR':DGR_CNTR.scores(), 'K_PTH_CNTR':K_PTH_CNTR.scores(),'KTZ_CNTR':KTZ_CNTR.scores(),
                'LPLC_CNTR':LPLC_CNTR.scores(), 'SP_EVZ':SP_EVZ.scores(),'HRM_CLSNS':HRM_CLSNS.scores() })
df.sample(8)

,Node,Est_BTWN,Aprx_BTWN,Aprx_CLSNS,Aprx_ELC_CLSNS,DGR_CNTR,K_PTH_CNTR,KTZ_CNTR,LPLC_CNTR,SP_EVZ,HRM_CLSNS
748,748,783.440026,0.0,0.000115,20.147167,613.0,94.466296,0.015349,1499996.0,0.015807,0.541647
1978,1978,9812.627814,0.0,0.000129,20.165840,1042.0,174.078582,0.023850,2912122.0,0.024574,0.609939
4071,4071,5.247950,0.0,0.000097,9.292003,18.0,3.387757,0.002931,42096.0,0.000435,0.429305
4248,4248,248.717784,0.0,0.000110,19.424029,262.0,52.510231,0.007367,435202.0,0.003861,0.501133
3217,3217,5.198415,0.0,0.000098,9.784034,18.0,5.602829,0.002927,41248.0,0.000427,0.430243
887,887,224.037588,0.0,0.000109,19.158008,256.0,39.480397,0.008032,580058.0,0.006665,0.492225
2939,2939,870.068766,0.0,0.000116,19.366804,439.0,82.739446,0.011499,971480.0,0.009975,0.529946
75,75,0.943192,0.0,0.000091,5.594349,8.0,2.215072,0.002673,9520.0,0.000118,0.404751


In [ ]:
df.to_csv(  path.join(features_save_path,'NODES FEATURES.csv') , index=False )

# 4 - Computing Distance between Nodes

## 4.1 - APSP Distance

In [ ]:
APSP_distance = nk.distance.APSP(Graph)
APSP_distance.run()
records = []
for u, v in tqdm(Graph.iterEdges() , total=Graph.numberOfEdges()):
    records.append( [int(u),int(v),APSP_distance.getDistance(u,v)] )

df = DataFrame.from_records(records , columns=['node_u','node_v','nk.distance.APSP'])
df.to_csv( path.join(features_save_path,'distance APSP.csv'),index=False)
df.sample(4)

  0%|          | 0/1067911 [00:00<?, ?it/s]

,node_u,node_v,nk.distance.APSP
623855,1073,1372,1.0
732146,1346,3033,1.0
170934,292,1252,1.0
1022314,3181,4243,1.0


# 5 - Saving Features

In [32]:
!zip -r Features\ -\ Networkit.zip Features\ -\ Networkit

  adding: Features - Networkit/ (stored 0%)


In [33]:
from google.colab import files
files.download('Features - Networkit.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>